# Calculation of all signatures

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd

import gc
import os
from glob import glob
from oggm import utils
from tqdm import tqdm
from tqdm.notebook import tqdm

os.chdir('/home/rooda/OGGM_results/')

import warnings
warnings.simplefilter('ignore', np.RankWarning)

In [ ]:
def preprocess(ds): 
    ds = ds.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month'])
    
    # hydro-variables 
    ds["total_runoff"] = (ds.melt_off_glacier + ds.melt_on_glacier + ds.liq_prcp_off_glacier + ds.liq_prcp_on_glacier)*1e-3 # m3/yr
    ds["total_runoff_monthly"] = ((ds.melt_off_glacier_monthly + ds.melt_on_glacier_monthly + ds.liq_prcp_off_glacier_monthly + ds.liq_prcp_on_glacier_monthly)*1e-3)/(30*86400) # in m3/s
    ds["melt_on_glacier"] = (ds.melt_on_glacier)*1e-3 # in # m3/yr
    ds["melt_on_glacier_monthly"] = ((ds.melt_on_glacier_monthly)*1e-3)/(30*86400) # in m3/s
    return ds[save_variables]

## Data

In [ ]:
RGI6_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6_v2.shp")
RGI6_ids = RGI6_ids[RGI6_ids.area_km2 > 1][["RGIId", "ID_basin"]]

RGI7_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI7_v2.shp")
RGI7_ids = RGI7_ids[RGI7_ids.area_km2 > 1]
RGI7_ids = utils.cook_rgidf(RGI7_ids, o1_region='17', o2_region='02', bgndate= RGI7_ids.src_date, 
                            version = "70", assign_column_values= {'Zone' : 'Zone', 'ID_basin' : 'ID_basin'})
RGI7_ids = RGI7_ids[["RGIId", "ID_basin"]]

# merge both datasets
ids = pd.concat([RGI6_ids, RGI7_ids]).set_index("RGIId")

In [ ]:
# area in m2
basins = gpd.read_file("zip:////home/rooda/Dropbox/Patagonia/MS2 Results/zenodo/basins_boundaries.zip")
basins["rgi_id"] = basins.basin_id.astype("int64") # to have the same ids with glaciers/catchments
basins = basins.set_index("rgi_id") 
basins = basins.basin_area.to_xarray() * 1e6 

In [ ]:
# remove unnecessary variables and coordinates
save_variables = ["melt_on_glacier", "melt_on_glacier_monthly", "total_runoff", "total_runoff_monthly"]

ref_period = slice("1980-01-01", "2014-12-31") 
future_period = slice("2070-01-01", "2099-12-31") 

# Calculation of all metrics

In [7]:
# Folder with all the results
gdirs = glob("/home/rooda/OGGM_results/runs/*", recursive = True)
gdirs = [a for a in gdirs if os.path.isdir(a)]

variables = ["melt_on_glacier", "total_runoff"]

ds_ref_magnitude        = []
ds_peak_water_year      = []
ds_peak_water_magnitude = []
ds_peak_water_duration  = []
ds_interannual_var      = []
ds_seasonal_cont        = []
ds_seasonal_var         = []
ds_seasonal_shift       = []
ds_lt_trend             = []
ds_lt_change            = []

for variable in tqdm(variables):
    for gdir in tqdm(gdirs):

        # read historical run 
        model_hist  = xr.open_dataset(glob(gdir + "/run_outputs_*.nc", recursive = False)[0])
        model_hist  = preprocess(model_hist)
        model_hist  = model_hist.drop_sel(time=[1980, 2020]) # check NAs

        paths = glob(gdir + "/run_output_*ssp*.nc", recursive = True)
        for path in tqdm(paths, leave = False):

            # read future run and concatenate
            model_future = xr.open_dataset(path)
            model_future = preprocess(model_future)
            model   = xr.concat([model_hist, model_future], dim = "time")

            # add basin ID to each glacier ID (RGI_ID)
            ids_subset = ids[ids.index.isin(model.rgi_id.to_pandas().tolist())]
            model = model.assign_coords(rgi_id = ids_subset.ID_basin.tolist())
            basin_area = basins[basins.rgi_id.isin(ids_subset.ID_basin)]
            
            # aggregate based on "new" ID
            model = model.groupby('rgi_id').sum()
            model = model.drop_sel(time=2099)

            # ID of the setup
            experiment_id = pd.Series(data = {'Variable': variable,
                                              'Outline': os.path.basename(gdir).split("_")[0],
                                              'Climate': os.path.basename(gdir).split("_")[1],
                                              'Volume':  os.path.basename(gdir).split("_")[2],
                                              'GCM':     os.path.basename(path).split("_")[2],
                                              'SSP':     os.path.basename(path).split("_")[3],
                                              'BCM':     os.path.basename(path).split("_")[4][0:3]})

            ## 1. Reference magnitude (mm yr-1)
            ref_magnitude = model[variable].sel(time = ref_period).mean(dim = "time").to_series()
            ref_magnitude = (ref_magnitude / basin_area) * 1e3 # from m to mm 
            ref_magnitude = pd.DataFrame(pd.concat([experiment_id, ref_magnitude]), columns=['ref_magnitude']).transpose()
            ds_ref_magnitude.append(ref_magnitude)
            
            # the following metrics are based on a 11-year moving average
            rolling = model[variable].rolling(time=11, center=True).mean()

            ## 2. Peak water year (years)
            peak_water_year = rolling.idxmax(dim = "time").astype("int16").to_series()      
            peak_water_year = pd.DataFrame(pd.concat([experiment_id, peak_water_year]), columns=['peak_water_year']).transpose()
            ds_peak_water_year.append(peak_water_year)

            ## 3. Peak water magnitude (mm yr-1)
            peak_water_magnitude = rolling.max(dim = "time")
            peak_water_magnitude = (peak_water_magnitude / basin_area) * 1e3 # from m to mm 
            peak_water_magnitude = peak_water_magnitude.to_series()      
            peak_water_magnitude = pd.DataFrame(pd.concat([experiment_id, peak_water_magnitude]), columns=['peak_water_magnitude']).transpose()
            ds_peak_water_magnitude.append(peak_water_magnitude)

            ## 4. Peak water duration (years)
            peak_water_duration = 0.9 * rolling.max(dim = "time")
            peak_water_duration = (rolling > peak_water_duration).sum(dim = "time").to_series()
            peak_water_duration = pd.DataFrame(pd.concat([experiment_id, peak_water_duration]), columns=['peak_water_duration']).transpose()
            ds_peak_water_duration.append(peak_water_duration)
            
            ## 5. Inter-annual variability (mm yr-1)
            interannual_var = model[variable] - rolling # detreting
            interannual_var = (interannual_var / basin_area) * 1e3 # from m to mm 
            interannual_var = interannual_var.std(dim = "time").to_series()
            interannual_var = pd.DataFrame(pd.concat([experiment_id, interannual_var]), columns=['interannual_var']).transpose()
            ds_interannual_var.append(interannual_var)
            
            ## 6. Seasonal contribution (%)
            seasonal_cont = model[variable + "_monthly"].sel(time = ref_period)
            seasonal_cont = seasonal_cont.mean(dim = "time")
            seasonal_cont = seasonal_cont.sel(month_2d = [12,1,2]).sum(dim = "month_2d") / seasonal_cont.sum(dim = "month_2d")
            seasonal_cont = seasonal_cont.to_series() * 100 # in percentage
            seasonal_cont = pd.DataFrame(pd.concat([experiment_id, seasonal_cont]), columns=['seasonal_cont']).transpose()
            ds_seasonal_cont.append(seasonal_cont)
            
            # 7. Seasonal variability (%)
            seasonal_var = model[variable + "_monthly"].sel(time = ref_period)
            seasonal_var = seasonal_var.sel(month_2d = [12,1,2]).sum(dim = "month_2d") / seasonal_var.sum(dim = "month_2d")
            seasonal_var = seasonal_var.std(dim = "time").to_series() * 100 # in percentage
            seasonal_var = pd.DataFrame(pd.concat([experiment_id, seasonal_var]), columns=['seasonal_var']).transpose()
            ds_seasonal_var.append(seasonal_var)
            
            ## 8. Seasonal shift (%)
            seasonal_shift = model[variable + "_monthly"]
            seasonal_shift = seasonal_shift.sel(month_2d = [12,1,2]).sum(dim = "month_2d") / seasonal_shift.sum(dim = "month_2d")
            seasonal_shift = seasonal_shift.sel(time = ref_period).mean(dim = "time") - seasonal_shift.sel(time = future_period).mean(dim = "time")
            seasonal_shift = seasonal_shift.to_series() * 100 # in percentage
            seasonal_shift = pd.DataFrame(pd.concat([experiment_id, seasonal_shift]), columns=['seasonal_shift']).transpose()
            ds_seasonal_shift.append(seasonal_shift)
            
            ## 9. Long-term trend (% dec-1)
            rolling_norm = rolling/rolling.max(dim = "time")  # normalize values (max = 1)
            rolling_norm = rolling_norm.transpose("rgi_id", "time") # trick to complete loop

            lt_trend = []
            for catchment in model.rgi_id.to_numpy(): # get the trend for each catchment

                time_period = slice(int(peak_water_year[catchment].iloc[0]), int(peak_water_year[catchment].iloc[0])+30)
                lt_trend_i = rolling_norm.sel(time = time_period, rgi_id = catchment)
                lt_trend_i = lt_trend_i.polyfit(dim = "time", deg = 1, skipna = True).polyfit_coefficients[0].to_numpy()
                lt_trend_i = lt_trend_i * 100 * 10 # final value: %% per decade
                lt_trend.append(lt_trend_i)

            lt_trend = pd.Series(lt_trend, index=model.rgi_id.to_numpy())
            lt_trend = pd.DataFrame(pd.concat([experiment_id, lt_trend]), columns=['lt_trend']).transpose()
            ds_lt_trend.append(lt_trend)
            
            ## 10. Long-term change
            lt_change = model[variable].sel(time = future_period).mean(dim = "time") / model[variable].sel(time = ref_period).mean(dim = "time")
            lt_change = (lt_change-1).to_series() * 100
            lt_change = pd.DataFrame(pd.concat([experiment_id, lt_change]), columns=['lt_change']).transpose()
            ds_lt_change.append(lt_change)
            
# concatenate all signatures 
dataset = pd.concat([pd.concat(ds_ref_magnitude),
                     pd.concat(ds_peak_water_year),
                     pd.concat(ds_peak_water_magnitude),
                     pd.concat(ds_peak_water_duration),
                     pd.concat(ds_interannual_var),
                     pd.concat(ds_seasonal_cont),
                     pd.concat(ds_seasonal_var),
                     pd.concat(ds_seasonal_shift),
                     pd.concat(ds_lt_trend),
                     pd.concat(ds_lt_change)])
                         
dataset.to_csv("/home/rooda/Dropbox/Patagonia/MS2 Results/dataset_hydro_signatures.csv")